# Hyazintharas • Milestone 3 • Implementation of Analysis of a Single Subject

Steps according to our milestone 2:

- Preamble: import used modules like `mne`, `mne-bids`, `numpy` as `np`, and `matplotlib.pyplot` as `plt`
- Input: load EEG data
- Filter: Apply a band pass filter to the loaded multi channel signal.
- Re-sampling: Apply `mne.filter.resample()` with `newFsHz=80`
- Epoching: -1 s … +1 s (relative to stimulus label)
- Re-referencing: global mean
    - may shift before epoching?
- Cleaning: rejection (based on subject, channel, and time)
    - optional: Do automatic rejection to minimise effort of milestone 4 implementation.
- Interpolation: Interpolate rejected channels
- ICA: Apply ICA component analysis
    - note: Do not apply FastICA, but choose another version based on lecture notes.
- Transform: Apply Morlet Wavelet Transform.
- Averaging: Perform Averaging over repetitions per condition.
- Filtering: Apply band pass filters to split to α (8-12 Hz) and β (3-7 Hz) bands.
- Analysis:
    - Difference between conditions
    - Decoding
    - Source space
    - Time frequency

In [1]:
# Preamble: import used modules 

# tools for importing the EEG signals and for first checkups
from mne_bids import BIDSPath, read_raw_bids, inspect_dataset

# EEG data processing tools
import mne

# math and array processing tools
import numpy as np

# Set Matplotlib to render figures using Qt. This allows for interactive figures in dedicated windows.
import matplotlib
matplotlib.use('qtagg')

# expand search path to additional directories:
# - with notebooks/ as working directory: Git repository root directory + src/ subdirectory
# - with Git root as working directory: src/
import sys
for newpath in ["../src", "src"]:
  sys.path.insert(0, newpath)

# custom modules for data loading and handling
import data_handling.data_downloader as dl
import data_handling.data_patcher as patch

## Input: Load and Prepare Data

In [3]:
# set the directory in which the dataset should be or shall get stored
dl.DATA_BASE_DIR = "./data/"

In [3]:
# check data, if missing or invalid download them, and if needed patch the file paths
dl.fetchData()
patch.patchAllFiles()

In [4]:
# set the dataset base path
bids_root = "./data/ds003702"

In [5]:
# load the current subjects data

# set the parameters
subject_id_int = 2  # Set the index of the current subject.
                    # Valid subject indices are given in the printed list of directories in the preious cell.
                    # Here, indices 1 to 50 seem to be valid.
subject_id_str = f"{subject_id_int:02d}"
datatype = "eeg"
suffix = "eeg"
task = "SocialMemoryCuing"

# generate the BIDS path object
bids_path = BIDSPath(
    subject=subject_id_str,
    task=task,
    datatype=datatype,
    suffix=suffix,
    root=bids_root
)

# TODO (optional): load re-sampled multi channel signal (pre-processed regarding band pass filter and resampling, smaller file size)
#  note: may use pickle or any other tool

In [6]:
# checkup: print the BIDS path objects content (path to the data file)
print(bids_path)

data/ds003702/sub-02/eeg/sub-02_task-SocialMemoryCuing_eeg.vhdr


In [7]:
# load the raw EEG data according to the previously generated BIDS path object
raw = read_raw_bids(bids_path) 

Extracting parameters from data/ds003702/sub-02/eeg/sub-02_task-SocialMemoryCuing_eeg.vhdr...
Setting channel info structure...
Reading channel info from data/ds003702/sub-02/eeg/sub-02_task-SocialMemoryCuing_channels.tsv.


/tmp/ipykernel_213430/2821730418.py:2: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = read_raw_bids(bids_path)
/tmp/ipykernel_213430/2821730418.py:2: RuntimeWarning: Did not find any events.tsv associated with sub-02_task-SocialMemoryCuing.

The search_str was "data/ds003702/sub-02/**/eeg/sub-02*events.tsv"
  raw = read_raw_bids(bids_path)
/tmp/ipykernel_213430/2821730418.py:2: RuntimeWarning: The number of channels in the channels.tsv sidecar file (59) does not match the number of channels in the raw data file (64). Will not try to set channel names.
  raw = read_raw_bids(bids_path)


In [8]:
# Filter
# Apply a band pass filter to the loaded multi channel signal.

f_sample_hz = 500 # see e.g. <bids_root>/sub-01/eeg/<…>.json -> "SamplingFrequency: 500"
f_cutoff_low_hz  =  1
f_cutoff_high_hz = 32

raw_filtered = mne.filter.filter_data( # alternative: raw.copy().filter(
    data=raw.get_data(), # return 2D (n_channels, n_times), or 3D (n_epochs, n_channels, n_times) array of data
    picks=None, # choose data for filtering; None: pick all channels, <list>: pick selected channels
    l_freq=f_cutoff_low_hz,
    h_freq=f_cutoff_high_hz,
    sfreq=f_sample_hz,
    filter_length="auto",
    method="iir", # alternatively, use "fir" for fixed duration of influence, but slower filter response
    iir_params=None, # default: 4 th order Butterworth filters; else: set parameters in dict()
    # n_jobs=None, # if cupy is installed and `method="fir"` is selected, `n_jobs="cuda"` is valid
    copy=False, # True: return filtered copy; False: operate in place
    verbose=True # logging level
)

# TODOs:
# - check, whether raw, raw.get_data(), or anything else needs to get passed for passing data to the filtering function
# - check, which other parameters should get passed
# - check, why the IIR filter design generates the Butterworth filter for -6 dB at both f_cutoff. This should be 50 % of the power and therefore -3 dB.
# - plot comparison of unfiltered and filtered signals (1-2 channels, each)

Setting up band-pass filter from 1 - 32 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 32.00 Hz: -6.02, -6.02 dB


In [9]:
# checkup: print shape of the filtered data object
# results: 2d array of shape [n channels x n frames]
raw_filtered.shape

(64, 1454787)

In [10]:
# Re-sampling for reducing computational effort and memory occupation

# set new sample rate in Hz
f_sample_new_hz = 80
downsampling_factor = f_sample_hz / f_sample_new_hz
print(f"Apply downsampling by a factor of {np.round(downsampling_factor, 2)} from {f_sample_hz} to {f_sample_new_hz}.")

# apply re-sampling
raw_resampled = mne.filter.resample(x=raw_filtered, down=downsampling_factor, npad="auto", axis=1) # TODO: set axis and other parameters

Apply downsampling by a factor of 6.25 from 500 to 80.


In [11]:
print(f"shape of the loaded signal with dimensions [n channels, n frames]:      {raw.get_data().shape}")
print(f"shape of the filtered signal with dimensions [n channels, n frames]:    {raw_filtered.shape}")
print(f"shape of the downsampled signal with dimensions [n channels, n frames]: {raw_resampled.shape}")

shape of the loaded signal with dimensions [n channels, n frames]:      (64, 1454787)
shape of the filtered signal with dimensions [n channels, n frames]:    (64, 1454787)
shape of the downsampled signal with dimensions [n channels, n frames]: (64, 232766)


In [12]:
# TODO (optional): store re-sampled multi channel signal to get smaller files in the next run
#  note: may use pickle or any other tool

In [13]:
# Epoching: -1 s … +1 s (relative to stimulus label)
#  note: see own solution

In [14]:
# Re-referencing: global mean
#  note: may shift this step to before epoching?
#  note: see own solution of prev. semester

In [15]:
# Cleaning: rejection (based on subject, channel, and time)
#  optional: Do automatic rejection to minimise effort of milestone 4 implementation.
#  note: see solution of example tasks for automatic rejection

In [16]:
# Interpolation: Interpolate rejected channels


In [17]:
# ICA: Apply ICA component analysis
    # note: Do not apply FastICA, but choose another version based on lecture notes.
    # note: for implementation, see prev. semester's implementation
    # note: may apply additional band pass filter for better results

In [18]:
# ICA: Apply denoising using ICA results
    # note: may apply on loaded signal with lower high-pass filter cutoff frequency

In [19]:
# Transform: Apply Morlet Wavelet Transform.


In [20]:
# Averaging: Perform Averaging over repetitions per condition.


In [21]:
# Filtering: Apply band pass filters to split to α (8-12 Hz) and β (3-7 Hz) bands.


In [22]:
# Analysis: Difference between conditions
    # note: include basic statistics

In [23]:
# Analysis: Decoding

In [24]:
# Analysis: Source space
    # note: plot computed location of the main effects

In [25]:
# Analysis: Time frequency